# 빅데이터 분석기사 실기 기출 2회 작업형 2 
[캐글 링크](https://www.kaggle.com/code/agileteam/tutorial-t2-2-python/notebook)  

## 전자상거래 배송 데이터
- 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
- 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측 확률값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
```
ID, Reached.on.Time_Y.N
4733,0.6
2040,0.8
5114,0.45
2361,0.23
5996,0.43
```


(유의사항)
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
- 수험번호.csv파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 ROC-AUC형태로 읽는다.

## 시험 환경처럼 진행

In [19]:
# # 시험환경 세팅 (코드 변경 X)
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split

# def exam_data_load(df, target, id_name="", null_name=""):
#     if id_name == "":
#         df = df.reset_index().rename(columns={"index": "id"})
#         id_name = 'id'
#     else:
#         id_name = id_name
    
#     if null_name != "":
#         df[df == null_name] = np.nan
    
#     X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
#     y_train = X_train[[id_name, target]]
#     X_train = X_train.drop(columns=[target])

    
#     y_test = X_test[[id_name, target]]
#     X_test = X_test.drop(columns=[target])
#     return X_train, X_test, y_train, y_test 
    
# df = pd.read_csv("C:/Users/36687/빅분기/작업형2/Data/2_train.csv")
# X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

# X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8799, 11), (2200, 11), (8799, 2), (2200, 2))

In [20]:
# # 제공된 3개 데이터 (y_test데이터 활용X)
# X_train.shape, X_test.shape, y_train.shape

((8799, 11), (2200, 11), (8799, 2))

In [40]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("C:/Users/36687/빅분기/작업형2/Data/2_train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 제공된 3개 데이터 (y_test데이터 활용X)
X_train.shape, X_test.shape, y_train.shape



# 0. 데이터 로드
print("train ======================================")
print(X_train.head(), "\n")
print("test =======================================")
print(X_test.head(), "\n")

print(y_train)

# 1 . EDA
# 1-1) train EDA
print(X_train.info(), "\n")
print(X_train.isnull().sum(), "\n")      # train, test 다 결측치 존재 X
print(X_train.describe(), "\n")

# 1-2) test EDA
print(X_test.info(), "\n")
print(X_test.isnull().sum(), "\n")
print(X_test.describe(), "\n")

# 1-3) train, test nunique 비교
print(X_train.describe(include="O"), "\n")     # 동일, 이상없음. pd.get_dummies 가자
print(X_test.describe(include="O"), "\n")



# 2. 전처리
# 2-1) 결측치 처리, id같은 불필요 칼럼 제거, X와 y 분리
# 결측치 존재 X

# train ID 제거
X_train = X_train.drop("ID", axis=1)

# test ID 따로 분리
test_id = X_test.pop("ID")

# y 분리
y = y_train["Reached.on.Time_Y.N"]

print("train ======================================")
print(X_train.head(), "\n")
print("test =======================================")
print(X_test.head(), "\n")
print(y.head(3), "\n")



# 2-2) 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 수치형 칼럼 선택
con_cols = X_train.select_dtypes(exclude="object").copy().columns
print("수치형 칼럼 : ", con_cols, "\n")

X_train[con_cols] = scaler.fit_transform(X_train[con_cols])
X_test[con_cols] = scaler.transform(X_test[con_cols])



# 2-3) 인코딩
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

X_test = X_test[X_train.columns] # 칼럼 순서 맞춰주기



print("train ======================================")
print(X_train.head(), "\n")
print("test =======================================")
print(X_test.head(), "\n")



# 3. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, random_state=2023, stratify=y)
print(X_train.shape, X_val.shape, "\n")




# 4. 모델링 : 분류
# 4-1) 성능지표
from sklearn.metrics import roc_auc_score

# 4-2) 랜덤포레스트 분류
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=2023)
rf.fit(X_train , y_train)

rf_pred_proba = rf.predict_proba(X_val)[:, 1]
rf_roc = roc_auc_score(y_val, rf_pred_proba)
print("랜덤 포레스트 roc : ", rf_roc)



# 4-3) lgbm 분류
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=2023)
lgbm.fit(X_train, y_train)

lgbm_pred_proba = lgbm.predict_proba(X_val)[:, 1]
lgbm_roc = roc_auc_score(y_val, lgbm_pred_proba)
print("lgbm roc : ", lgbm_roc)


# # 하이퍼 파라미터 튜닝
# from sklearn.model_selection import GridSearchCV

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=2023)

# from lightgbm import LGBMClassifier
# lgbm = LGBMClassifier(random_state=2023)

# models = [rf, lgbm]

# params = {"max_depth" : [1, 2, 3], "n_estimators" : [100, 200, 300]}

# for model in models : 
# 	gs = GridSearchCV(model, param_grid= params, cv=5, scoring="roc_auc", n_jobs=4)
# 	gs.fit(X_train, y_train)
	
# 	print("="*10)
# 	print(f"model : {model}")
# 	print(f"best parameter : {gs.best_params_}")
# 	print(f"Train score : {gs.score(X_train, y_train)}")
# 	print(f"best score : {gs.best_score_}")



# 5. 제출 : df, csv
# 최종 : rf pred
pred = rf.predict_proba(X_test)[:, 1]

submit = pd.DataFrame({"ID" : test_id, "Reached.on.Time_Y.N" : pred})
submit.to_csv("2_submission.csv", index=False)

check = pd.read_csv("2_submission.csv")
print(check.head())

train ======================================
        ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
3999  4000               B             Ship                    3   
9559  9560               F             Ship                    4   
2649  2650               B             Ship                    2   
4843  4844               F             Ship                    6   
9601  9602               F           Flight                    3   

      Customer_rating  Cost_of_the_Product  Prior_purchases  \
3999                4                  134                3   
9559                3                  173                3   
2649                1                  192                3   
4843                5                  284                4   
9601                1                  246                3   

     Product_importance Gender  Discount_offered  Weight_in_gms  
3999               high      F                 6           5680  
9559             medium      M     

- 기본 random_state만
```
랜덤 포레스트 roc :  0.7599943673146259
lgbm roc :  0.7491514479693568
````

- 그리드 서치 결과
```
==========
model : RandomForestClassifier(random_state=2023)
best parameter : {'max_depth': 3, 'n_estimators': 100}
Train score : 0.7635182991383354
best score : 0.7451674489696241
==========
model : LGBMClassifier(random_state=2023)
best parameter : {'max_depth': 2, 'n_estimators': 100}
Train score : 0.7790380739874672
best score : 0.7460478258739531
```